## **Modelo de Clasificación**
El objetivo de este ejercicio es encontrar un modelo que permita predecir si los pasajeros del titanic sobreviven o no.


Descripción del dataSet:

* survival - Survival (0 = No; 1 = Yes)
* class - Passenger Class (1 = 1st; 2 = 2nd; 3 = 3rd)
* name - Name
* sex - Sex
* age - Age
* sibsp - Number of Siblings/Spouses Aboard
* parch - Number of Parents/Children Aboard
* ticket - Ticket Number
* fare - Passenger Fare
* cabin - Cabin
* embarked - Port of Embarkation (C = Cherbourg; Q = Queenstown; S = * Southampton)
* boat - Lifeboat (if survived)
* body - Body number (if did not survive and body was recovered)


Utilizaremos los modelos de clasificación RandomForest, Regresion Logística,  y clasificación MultiClass. 

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html

Realizemos drop de las filas con na o null
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html


Ayuda:

1. Tratamiento de nulos
En este ejercicio eliminaremos todas las filas que tengan algun valor nulo mediante

data_selected = data_selected.dropna(axis=0)


El parámetro axis=0 indica que se eliminaran las filas. Si se utiliza axis=1, se elimina la columna completa.

Ver el tutorial de tratamiento de nulos del Kaggle.

[Tutorial Kaggle](https://www.kaggle.com/dansbecker/handling-missing-values)






In [3]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics




## Lectura de datos y exploración

Leer el fichero de datos "College.csv". 

In [4]:
file_path = "titanic.csv"
data = pd.read_csv(file_path)

Listar las primeras filas del fichero para ver los datos

In [5]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


¿cuantas filas y columnas tienen los datos?


In [6]:
data.shape

(891, 12)

Seleccione de los datos, las columnas con las que trabajaremos:
['Survived',
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked'])

In [7]:
selected = ['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
data_selected = data[selected]

Elimine las filas que tienen datos en nulos o na:

In [9]:
data_selected = data_selected.dropna()

Explore la cantidad de registros, la media, el desvio y los cuartiles de las columnas numéricas de los datos con que trabajaremos.

In [10]:
data_selected.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,712.000000,712.000000,712.000000,712.000000,712.000000,712.000000
mean,0.404494,2.240169,29.642093,0.514045,0.432584,34.567251
std,0.491139,0.836854,14.492933,0.930692,0.854181,52.938648
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,20.000000,0.000000,0.000000,8.050000
50%,0.000000,2.000000,28.000000,0.000000,0.000000,15.645850
75%,1.000000,3.000000,38.000000,1.000000,1.000000,33.000000
max,1.000000,3.000000,80.000000,5.000000,6.000000,512.329200


Liste las columnas del dataframe indicando el tipo de datos.

In [12]:
for col in data_selected.columns:
    print(f"Columnas een {col}: {data[col].dtype}")

Columnas een Survived: int64
Columnas een Pclass: int64
Columnas een Sex: object
Columnas een Age: float64
Columnas een SibSp: int64
Columnas een Parch: int64
Columnas een Fare: float64
Columnas een Embarked: object


¿Cuantas han sobrevido y cuantos no?

In [13]:
data_selected.groupby('Survived').size()

Survived
0    424
1    288
dtype: int64

## Preparación de los datos

Liste las variables categóricas 

In [16]:
categorical_cols = [col for col in data_selected.columns]
print(categorical_cols)

['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']


In [17]:

print("Valores únicos de 'Sex':", set(data['Sex']))
print("Valores únicos de 'Embarked':", set(data['Embarked']))

Valores únicos de 'Sex': {'male', 'female'}
Valores únicos de 'Embarked': {'Q', 'S', nan, 'C'}


Liste los diferentes valores de las columnas "categorical"

In [18]:

print("Valores únicos de 'Sex':", set(data['Sex']))
print("Valores únicos de 'Embarked':", set(data['Embarked']))

Valores únicos de 'Sex': {'male', 'female'}
Valores únicos de 'Embarked': {'Q', 'S', nan, 'C'}


Indique en una lista las columnas a las que aplicar "label encoder" en good_label_cols y las que no se aplicará el encoder en bad_label_cols

In [19]:
good_label_cols = ['Sex', 'Embarked']
bad_label_cols = [col for col in data_selected.columns if col not in good_label_cols and col != 'Survived']
print("good_label_cols:", good_label_cols)
print("bad_label_cols:", bad_label_cols)

good_label_cols: ['Sex', 'Embarked']
bad_label_cols: ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']


Aplique la función de labelEncoder a las label_cols. Liste las primeras filas resultantes del conjunto de datos "label_data"

In [20]:
data_selected.head()    


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


Defina la columna objetivo en col_y y en features todas las columnas de data_selected, excepto la columna objetivo

In [21]:
col_y = 'Survived'
features = [x for x in data_selected.columns if x != col_y]
features

['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']

Defina "y", y X. 

In [22]:
# target object y
y = data_selected[col_y]
# Create X
X = data_selected[features]

print(X.head())

   Pclass  Sex   Age  SibSp  Parch     Fare  Embarked
0       3    1  22.0      1      0   7.2500         2
1       1    0  38.0      1      0  71.2833         0
2       3    0  26.0      0      0   7.9250         2
3       1    0  35.0      1      0  53.1000         2
4       3    1  35.0      0      0   8.0500         2


Defina las particiones para entrenamiento y de validación con semila=1 y dejando un 20% de filas para test. Imprima cuantas filas utilizará para entrenamiento y cuantas para validar

In [23]:
# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.2, random_state=1)

In [24]:
print(train_X.count())


Pclass      569
Sex         569
Age         569
SibSp       569
Parch       569
Fare        569
Embarked    569
dtype: int64


In [ ]:
print(<FILL>)

OK


## Definición del modelo

Defina el modelo utilizando LogisticRegression con semilla=1

In [26]:
titanic = LogisticRegression(random_state=1)




Entrene el modelo. Aumente el numero de iteraciones si obtiene un error.

In [27]:
# Fit Model
titanic.fit(train_X, train_y)

c:\Users\nichi\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=1)

## Predicción y validación del modelo 

Utilice el modelo para realizar las predicciones con los datos de validación y calcule el valor "score" o "accuracy" 

In [28]:
from sklearn import metrics
val_predictions = titanic.predict(val_X)
score = round(titanic.score(val_X, val_y), 2)
print("Score datos validación %f" % score)
print("Accuracy:", metrics.accuracy_score(val_y, val_predictions))

Score datos validación 0.760000
Accuracy: 0.7622377622377622


In [29]:
assert round(score*100)==76,"Error en resultado de score"

Calcule los resultados utilizando random forest con 1000 estimadores y max_depth=10, y semilla=1

In [30]:
RF = RandomForestClassifier(n_estimators=1000, max_depth=10, random_state=1)
RF.fit(train_X, train_y)
RF.predict(val_X)
round(RF.score(val_X, val_y), 2)


0.81

Calcule los resultados del modelo utilizando Multi-Class Classification utilizando solver='lbfgs' y un max_iter=1000

In [31]:
NN = MLPClassifier(solver='lbfgs', max_iter=1000)
NN.fit(train_X, train_y)
NN.predict(val_X)
round(NN.score(val_X, val_y), 2)

c:\Users\nichi\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


0.76

## Resultados finales

Utilice el mejor modelo obtenido y calcule el score con todos los datos 

In [32]:
RF.predict(X)
score = round(RF.score(X, y), 2)
round(score, 2)


0.93

In [33]:
assert round(score,2)*100==93,"Error en resultado de porcentaje de error"